## Import Relevant Packages

In [2]:
import os
import geopandas as gpd
import cartopy.crs as ccrs
import folium

## Import Data

In [4]:
substations = gpd.read_file("../Data/belfastSubstations.shp") # Import Substations data from Data folder within Repository
EVcharging = gpd.read_file("../Data/evChargingStations_belfast.shp") # Import EV Charging data from same folder

## Reproject Data for .buffer

In [6]:
substations_wm = substations.to_crs(epsg=3857) # Convert to Web Mercator
EVcharging_wm = EVcharging.to_crs(epsg=3857) # Convert to Web Mercator

## Create Buffer

In [8]:
buffer = substations_wm.buffer(200) # Create a 200m buffer around the Belfast Substations
buffer_gdf = gpd.GeoDataFrame(geometry=buffer) # Convert the GeoSeries into a GeoDataFrame
buffer_gdf.set_crs(epsg=3857, inplace=True) # Sets GDF to EPSG 4326

buffer_gdf.head() # Show the first 5 records within the newly created GeoDataFrame

,geometry
0,"POLYGON ((-655140.927 7283955.987, -655141.89 ..."
1,"POLYGON ((-663618.734 7288509.934, -663619.697..."
2,"POLYGON ((-665331.483 7277942.59, -665332.446 ..."
3,"POLYGON ((-660890.514 7282194.005, -660891.477..."
4,"POLYGON ((-658522.626 7283962.461, -658523.589..."


## Classify Charging Points

In [10]:
within_buffer = [] # Create an empty list to store inside or outside values

for point in EVcharging_wm.geometry: # Loop through all points within the EV Charging Points data
    is_within = buffer_gdf.intersects(point).any() # Check if point intersects buffer
    within_buffer.append(is_within) # Append result within list

EVcharging_wm["within_buffer"] = within_buffer # Adds list as column to dataset

## Create an inside and outside GeoDataFrame

In [12]:
EV_inside = EVcharging_wm[EVcharging_wm["within_buffer"]] # Create a gdf for points within a 200m buffer of a substation 
EV_outside = EVcharging_wm[~EVcharging_wm["within_buffer"]] # Create a gdf for points outside the 200m buffers

EV_inside.head()

,chargeDevi,reference,name,latitude,longitude,subBuildin,buildingNa,buildingNu,thoroughfa,street,...,connect_73,connect_74,connect_75,connect_76,connect_77,connect_78,connect_79,connect_80,geometry,within_buffer
81,cc9657884708170e160c8372d92f3535,SC21,"Lower Crescent, Belfast (Socket 2)",54.587893,-5.934377,None,None,None,Lower Crescent,None,...,None,None,None,None,None,None,None,None,POINT (-660611.826 7282291.865),True
82,d6bdad12e64bc74082fb649e46620eab,SC21-A,Lower Crescent,54.587940,-5.933900,None,None,None,None,Lower Crescent,...,None,None,None,None,None,None,None,None,POINT (-660558.726 7282300.895),True
83,52832a78d44e649dd9a15d87c39feb9d,SC21-B,Lower Crescent,54.587940,-5.933900,None,None,None,None,Lower Crescent,...,None,None,None,None,None,None,None,None,POINT (-660558.726 7282300.895),True


## Reproject data for folium

In [14]:
buffer_f = buffer_gdf.to_crs(epsg=4326) # Convert buffer data to EPSG:4326
substations_f = substations_wm.to_crs(epsg=4326) # Convert substation data to EPSG:4326
EVinside_f = EV_inside.to_crs(epsg=4326) # Convert EV points inside buffer to EPSG:4326
EVoutside_f = EV_outside.to_crs(epsg=4326) # Convert EV points outside buffer to EPSG:4326

## Create Folium Map

In [16]:
m = buffer_f.explore(color="lightsalmon", popup=False, legend=True) # Create a folium map with buffer GDF added

## Add Additional Data

In [18]:
substations_f.explore( m=m, # Add to existing map
                      marker_type="circle_marker", # Represent as small circle 
                      color="dodgerblue", # Colour blue
                      popup=True, # Enable popup
                      legend=True) # Show on Legend

In [19]:
for idx, row in EVinside_f.iterrows(): # Loop through each row in EVinside_f
    folium.Marker(
        location=[row.geometry.y, row.geometry.x], # Set marker location using point coordinates from GDF
        popup=str(row), # Convert row to a string for the popup
        icon=folium.Icon(
        color='green', # Set marker colour to green
        icon='car', # Select car icon
        prefix='fa' # Specify source of icon as FontAwesome ('fa')
        )
    ).add_to(m) # Add marker to the existing map


for idx, row in EVoutside_f.iterrows(): # Loop through each row in EV_outside_f
    folium.Marker(
        location=[row.geometry.y, row.geometry.x], # Set marker location using point coordinates from GDF
        popup=str(row), # Convert row to a string for the popup
        icon=folium.Icon(
        color='red', # Set marker colour to red
        icon='car', # Select car icon
        prefix='fa' # Specifiy source of icon as FontAwesome ('fa')
        )
    ).add_to(m) # Add marker to the existing map

m

## Save link

In [21]:
m.save("EVcharging_Belfast.html")